In [1]:
import pandas as pd
import numpy as np 
import matplotlib 
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsClassifier
%matplotlib inline

### Loading the data 

In [2]:
data = pd.read_csv('Data/data.csv')
pd.set_option('display.max_columns', None)
data.shape

(119011, 34)

In [3]:
data.tail()

,GAME_ID,MATCHUP,LOCATION,W,FINAL_MARGIN,SHOT_NUMBER,PERIOD,GAME_CLOCK,SHOT_CLOCK,DRIBBLES,TOUCH_TIME,SHOT_DIST,PTS_TYPE,SHOT_RESULT,CLOSEST_DEFENDER,CLOSEST_DEFENDER_PLAYER_ID,CLOSE_DEF_DIST,FGM,PTS,PLAYER_NAME,PLAYER_ID,PLAYER_HEIGHT,DEFENDER_HEIGHT,PLAYER_PTS,PLAYER_FG%,PLAYER_3P%,PLAYER_FT%,PLAYER_AST,PLAYER_WEIGHT,DEFENDER_OREB,DEFENDER_DREB,DEFENDER_STL,DEFENDER_BLK,DEFENDER_WEIGHT
119006,21400175,"NOV 21, 2014 - MIL @ TOR",A,L,-42,3,4,8:55,15.2,0,0.8,22.9,3,missed,BRUNO CABOCLO,203998,10.6,0,0,JARED DUDLEY,201162,197.5,202.5,7.2,46.8,38.5,71.6,1.8,101.25,0.1,0.1,0.0,0.1,98.10
119007,21400003,"OCT 28, 2014 - HOU @ LAL",A,W,18,3,3,1:35,3.3,1,2.4,6.7,2,made,JULIUS RANDLE,203944,5.3,1,2,KOSTAS PAPANIKOLAOU,203123,200.0,202.5,4.2,35.0,29.2,72.2,2.0,101.25,0.0,0.0,0.0,0.0,112.50
119008,21400003,"OCT 28, 2014 - HOU @ LAL",A,W,18,2,2,10:10,24.0,0,0.3,4.1,2,made,JULIUS RANDLE,203944,3.2,1,2,DWIGHT HOWARD,2730,207.5,202.5,15.8,59.3,50.0,52.8,1.2,119.25,0.0,0.0,0.0,0.0,112.50
119009,21400744,"FEB 05, 2015 - SAC vs. DAL",H,L,-23,9,4,3:21,13.3,5,5.4,21.9,2,missed,RICKY LEDO,203495,5.0,0,0,NIK STAUSKAS,203917,195.0,201.0,4.4,36.5,32.2,85.9,0.9,92.25,0.9,1.2,0.4,0.1,88.00
119010,21400498,"JAN 03, 2015 - MIN vs. UTA",H,L,-12,7,4,0:26,2.6,1,0.7,4.3,2,made,TOURE' MURRY,203315,2.8,1,2,ANTHONY BENNETT,203461,200.0,192.5,5.2,42.1,30.4,64.1,0.8,110.25,0.2,0.0,0.2,0.0,87.75


In [4]:
data.columns.values

array(['GAME_ID', 'MATCHUP', 'LOCATION', 'W', 'FINAL_MARGIN',
       'SHOT_NUMBER', 'PERIOD', 'GAME_CLOCK', 'SHOT_CLOCK', 'DRIBBLES',
       'TOUCH_TIME', 'SHOT_DIST', 'PTS_TYPE', 'SHOT_RESULT',
       'CLOSEST_DEFENDER', 'CLOSEST_DEFENDER_PLAYER_ID', 'CLOSE_DEF_DIST',
       'FGM', 'PTS', 'PLAYER_NAME', 'PLAYER_ID', 'PLAYER_HEIGHT',
       'DEFENDER_HEIGHT', 'PLAYER_PTS', 'PLAYER_FG%', 'PLAYER_3P%',
       'PLAYER_FT%', 'PLAYER_AST', 'PLAYER_WEIGHT', 'DEFENDER_OREB',
       'DEFENDER_DREB', 'DEFENDER_STL', 'DEFENDER_BLK', 'DEFENDER_WEIGHT'],
      dtype=object)

### Manipulating the data

In [3]:
# separate data into matrix of predictor variables and vector of target variables
names = ['LOCATION', 'SHOT_NUMBER', 'PERIOD', 'GAME_CLOCK', 'SHOT_CLOCK',
         'DRIBBLES', 'SHOT_DIST', 'PTS_TYPE', 'CLOSE_DEF_DIST', 'FGM', 'PTS',
         'PLAYER_HEIGHT','DEFENDER_HEIGHT', 'PLAYER_PTS', 'PLAYER_FG%', 'PLAYER_3P%',
          'PLAYER_FT%','DEFENDER_STL', 'DEFENDER_BLK', 'SHOT_RESULT']

# converting data to usable numerical data
# todo: move this to preprocessing 
def convert_time(time_string):
    time = time_string.split(':')
    return float(time[0])*60 + float(time[1])

# convert game_clock to seconds
data['GAME_CLOCK'] = data['GAME_CLOCK'].apply(lambda x: convert_time(x))

# convert shot_result to either 0 or 1
data['SHOT_RESULT'] = (data['SHOT_RESULT'] == 'made').astype(int)
# convert win to either 0 

# encode categorical data 
data_encoded = pd.get_dummies(data[names])

# split into train and test set with roughly 75-25 split
train, test = train_test_split(data_encoded, test_size=0.25)
Xtrain = train.drop(['SHOT_RESULT'], axis=1)
ytrain = train['SHOT_RESULT']
Xtest = test.drop(['SHOT_RESULT'], axis=1)
ytest = test['SHOT_RESULT']

### Setting the baseline

First baseline we consider is to predict every shot within 5 feet of the basket as a make and anything else as a miss. 

In [6]:
yhat = (Xtest['SHOT_DIST'] <= 5).astype(int)
acc = np.mean(yhat == ytest)
print('Accuracy = {0:f}'.format(acc))

Accuracy = 0.607771


A more complex baseline we can try is to use k-nn algorithm. First, we need to normalize the data so that no one parameter exerts more influence on the result than others. 

In [7]:
# without normalizing data 
neigh = KNeighborsClassifier(n_neighbors=20)
neigh.fit(Xtrain,ytrain)

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
                     metric_params=None, n_jobs=None, n_neighbors=20, p=2,
                     weights='uniform')

In [8]:
yhat_ts = neigh.predict(Xtest)
acc = np.mean(ytest.values == yhat_ts)
print('Accuracy = {0:f}'.format(acc))

Accuracy = 0.637179


In [9]:
# with normaliziation 
scaler = StandardScaler()
scaler.fit(Xtrain)
Xtrain_norm = scaler.transform(Xtrain)
scaler.fit(Xtest)
Xtest_norm = scaler.transform(Xtest)

neigh = KNeighborsClassifier(n_neighbors=20)
neigh.fit(Xtrain_norm,ytrain)

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
                     metric_params=None, n_jobs=None, n_neighbors=20, p=2,
                     weights='uniform')

In [ ]:
yhat_ts_norm = neigh.predict(Xtest_norm)
acc = np.mean(ytest.values == yhat_ts_norm)
print('Accuracy = {0:f}'.format(acc))

In [ ]:
# finding out why the accuracy is so high 
Xtrain[Xtrain['FGM'] >= 1]

In [ ]:
# FGM is not the total field goals made, but whether or not that 
# individual shot was made or not, so we need to drop this 
Xtrain_new = Xtrain.drop(['FGM','PTS'], axis=1)
Xtest_new = Xtest.drop(['FGM', 'PTS'], axis=1)
scaler_new = StandardScaler()
scaler_new.fit(Xtrain_new)
Xtrain_norm_new = scaler_new.transform(Xtrain_new)
scaler_new.fit(Xtest_new)
Xtest_norm_new = scaler_new.transform(Xtest_new)

In [ ]:
neigh = KNeighborsClassifier(n_neighbors=20)
neigh.fit(Xtrain_norm_new,ytrain)

In [ ]:
yhat_ts_norm = neigh.predict(Xtest_norm_new)
acc = np.mean(ytest.values == yhat_ts_norm)
print('Accuracy = {0:f}'.format(acc))

In [ ]:
# trying different k's
neigh = KNeighborsClassifier(n_neighbors=10)
neigh.fit(Xtrain_norm_new,ytrain)
yhat_ts_norm = neigh.predict(Xtest_norm_new)
acc = np.mean(ytest.values == yhat_ts_norm)
print('Accuracy for k = 10: {0:f}'.format(acc))

neigh = KNeighborsClassifier(n_neighbors=30)
neigh.fit(Xtrain_norm_new,ytrain)
yhat_ts_norm = neigh.predict(Xtest_norm_new)
acc = np.mean(ytest.values == yhat_ts_norm)
print('Accuracy for k = 30: {0:f}'.format(acc))

neigh = KNeighborsClassifier(n_neighbors=50)
neigh.fit(Xtrain_norm_new,ytrain)
yhat_ts_norm = neigh.predict(Xtest_norm_new)
acc = np.mean(ytest.values == yhat_ts_norm)
print('Accuracy for k = 50: {0:f}'.format(acc))

neigh = KNeighborsClassifier(n_neighbors=100)
neigh.fit(Xtrain_norm_new,ytrain)
yhat_ts_norm = neigh.predict(Xtest_norm_new)
acc = np.mean(ytest.values == yhat_ts_norm)
print('Accuracy for k = 100: {0:f}'.format(acc))

Accuracy increases as k increases but the increases get smaller as k gets larger and larger. Let's try manually assigning weights to normalized features now and transforming features 

### Visualizing relationship between features and output

The goal is to see if there are trends between the features and the outcome. This is so we can manually input some weights for the KNN method to increase the baseline accuracy

In [ ]:
y = ytrain.values
make = (y==1)
miss = (y==0)

def plot_cnt(X,y,labels,ylim=None,xlim=None,start=0,end=-1):
    
    # add random "jitter" to seperate points
    var = .15;
    Xnew = X[start:end]
    makenew = make[start:end]
    missnew= miss[start:end]
    Xj = Xnew + var*np.random.randn(Xnew.shape[0],Xnew.shape[1]) 

    # Create the scatter plot
    plt.plot(Xj[makenew,0],Xj[makenew,1],'b.')
    plt.plot(Xj[missnew,0],Xj[missnew,1],'g.')
    plt.xlabel(labels[0], fontsize=16)
    plt.ylabel(labels[1], fontsize=16)
    plt.ylim(ylim)
    plt.xlim(xlim)
    plt.legend(['make','miss'],loc='upper right')
    return plt

In [ ]:
xnames1 = ['SHOT_DIST', 'DEFENDER_BLK']
X1 = Xtrain[xnames1].values

plot_cnt(X1,y,xnames1,ylim=(0,3.5),end=3000)

In [ ]:
plot_cnt(X1,y,xnames1,ylim=(0,3.5),start=3000,end=6000)

In [ ]:
plot_cnt(X1,y,xnames1,ylim=(0,3.5),start=6000,end=9000)

In [ ]:
xnames2 = ['SHOT_DIST', 'CLOSE_DEF_DIST']
X2 = Xtrain[xnames2].values

In [ ]:
plot_cnt(X2,y,xnames2,ylim=(0,12),end=3000)

In [ ]:
plot_cnt(X2,y,xnames2,ylim=(0,12),start=3000,end=6000)

In [ ]:
plot_cnt(X2,y,xnames2,ylim=(0,12),start=6000,end=9000)

In [ ]:
plot_cnt(X2,y,xnames2,ylim=(0,12),start=9000,end=12000)

In [ ]:
xnames3 = ['SHOT_DIST', 'SHOT_CLOCK']
X3 = Xtrain[xnames3].values

plot_cnt(X3,y,xnames3,ylim=(0,25),end=3000)

In [ ]:
plot_cnt(X3,y,xnames3,ylim=(0,25),start=3000,end=6000)

In [ ]:
plot_cnt(X3,y,xnames3,ylim=(0,25),start=6000,end=9000)

In [ ]:
xnames4 = ['SHOT_DIST', 'PLAYER_FG%']
X4 = Xtrain[xnames4].values

plot_cnt(X4,y,labels=xnames4,ylim=(30,70),end=3000)

In [ ]:
xnames5 = ['CLOSE_DEF_DIST', 'DEFENDER_HEIGHT']
X5 = Xtrain[xnames5].values

plot_cnt(X5,y,xnames5,xlim=(0,20),end=2000)

In [ ]:
# feature transformations
Xtrain_new['SHOT_DST/DEF_BLKS'] = \
    Xtrain_new['SHOT_DIST']/Xtrain_new['DEFENDER_BLK']

Xtest_new['SHOT_DST/DEF_BLKS'] = \
    Xtest_new['SHOT_DIST']/Xtest_new['DEFENDER_BLK']

In [ ]:
Xtrain_new[Xtrain_new['DEF_BLKS/SHOT_DIST'] == np.inf]

In [ ]:
Xtrain_new = Xtrain_new.replace([np.inf, -np.inf],0)
Xtest_new = Xtest_new.replace([np.inf, -np.inf],0)

In [ ]:
Xtrain_new

In [ ]:
scaler_new = StandardScaler()
scaler_new.fit(Xtrain_new)
Xtrain_norm_new = scaler_new.transform(Xtrain_new)
scaler_new.fit(Xtest_new)
Xtest_norm_new = scaler_new.transform(Xtest_new)

In [ ]:
neigh1 = KNeighborsClassifier(n_neighbors=100)
neigh1.fit(Xtrain_norm_new,ytrain)

In [ ]:
yhat_ts_norm = neigh1.predict(Xtest_norm_new)
acc = np.mean(ytest.values == yhat_ts_norm)
print('Accuracy: {0:f}'.format(acc))

Very small increase in accuracy. Let's manually add weights. These weights are a guess on how much a certain feature will play into determining whether a shot is a miss or make. This is just from years of watching and playing basketball. 

In [ ]:
# increase weight of player PPG by 5 
Xtrain_norm_new[:,10] = Xtrain_norm_new[:,10] * 5

# increase weight of player FG% by 5
Xtrain_norm_new[:,11] = Xtrain_norm_new[:,11] * 5

# increase weight of player 3P% by 2
Xtrain_norm_new[:,12] = Xtrain_norm_new[:,12] * 2

# increase weight of closest defender distance by 3
Xtrain_norm_new[:,7] = Xtrain_norm_new[:,7] * 3

# increase weight of shot distance by 2
Xtrain_norm_new[:,5] = Xtrain_norm_new[:,5] * 2

In [ ]:
# do the same for Xtest
Xtest_norm_new[:,10] = Xtest_norm_new[:,10] * 5
Xtest_norm_new[:,11] = Xtest_norm_new[:,11] * 5
Xtest_norm_new[:,12] = Xtest_norm_new[:,12] * 2
Xtest_norm_new[:,7] = Xtest_norm_new[:,7] * 3
Xtest_norm_new[:,5] = Xtest_norm_new[:,5] * 2

In [ ]:
Xtrain_norm_new[0]

In [ ]:
neigh1 = KNeighborsClassifier(n_neighbors=100)
neigh1.fit(Xtrain_norm_new,ytrain)

In [ ]:
yhat_ts_norm = neigh1.predict(Xtest_norm_new)
acc = np.mean(ytest.values == yhat_ts_norm)
print('Accuracy: {0:f}'.format(acc))

Accuracy went down a bit. It seems that manually setting these weights is difficult, let's try using a neural network to learn these weights/biases/features automatically 

In [4]:
import tensorflow as tf
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.layers import Dense, Activation 
import tensorflow.keras.backend as K
from tensorflow.keras import optimizers

In [5]:
ytrain = ytrain.values
ytest = ytest.values

In [6]:
# renormalize test and train sets
Xtr_new = Xtrain.drop(['FGM','PTS'], axis=1)
Xts_new = Xtest.drop(['FGM', 'PTS'], axis=1)
scaler_new = StandardScaler()
scaler_new.fit(Xtr_new)
Xtr_norm = scaler_new.transform(Xtr_new)
scaler_new.fit(Xts_new)
Xts_norm = scaler_new.transform(Xts_new)

In [7]:
K.clear_session()

In [ ]:
Xtr_new.shape

In [8]:
nin = Xtr_new.shape[1] # number of inputs 
nout = 1 # make or miss

In [9]:
# model with two hidden layers with 4 hidden units each
# using sigmoid activation 
model1S = Sequential()
model1S.add(Dense(units=4, input_shape=(nin,), activation='sigmoid', name='hidden1'))
model1S.add(Dense(units=4, input_shape=(nin,), activation='sigmoid', name='hidden2'))
model1S.add(Dense(units=nout, activation='sigmoid', name='output'))

In [10]:
model1S.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
hidden1 (Dense)              (None, 4)                 76        
_________________________________________________________________
hidden2 (Dense)              (None, 4)                 20        
_________________________________________________________________
output (Dense)               (None, 1)                 5         
Total params: 101
Trainable params: 101
Non-trainable params: 0
_________________________________________________________________


In [11]:
opt = optimizers.Adam(lr=0.001)
model1S.compile(optimizer=opt,
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [ ]:
model1S.fit(Xtr_norm, ytrain, epochs=30, batch_size=100, validation_data=(Xts_norm,ytest))

Train on 89258 samples, validate on 29753 samples
Epoch 1/30
89258/89258 [==============================] - 10s 113us/sample - loss: 0.6826 - accuracy: 0.5610 - val_loss: 0.6749 - val_accuracy: 0.5942
Epoch 2/30
89258/89258 [==============================] - 10s 116us/sample - loss: 0.6690 - accuracy: 0.6010 - val_loss: 0.6654 - val_accuracy: 0.6078
Epoch 3/30
89258/89258 [==============================] - 9s 102us/sample - loss: 0.6630 - accuracy: 0.6068 - val_loss: 0.6615 - val_accuracy: 0.6104
Epoch 4/30
89258/89258 [==============================] - 9s 96us/sample - loss: 0.6605 - accuracy: 0.6091 - val_loss: 0.6597 - val_accuracy: 0.6121
Epoch 5/30
89258/89258 [==============================] - 9s 105us/sample - loss: 0.6592 - accuracy: 0.6106 - val_loss: 0.6588 - val_accuracy: 0.6137
Epoch 6/30
89258/89258 [==============================] - 10s 111us/sample - loss: 0.6582 - accuracy: 0.6105 - val_loss: 0.6585 - val_accuracy: 0.6136
Epoch 7/30
89258/89258 [========================

In [ ]:
# model with two hidden layers with 8 hidden units each
# using sigmoid activation 
model2S = Sequential()
model2S.add(Dense(units=8, input_shape=(nin,), activation='sigmoid', name='hidden1'))
model2S.add(Dense(units=8, input_shape=(nin,), activation='sigmoid', name='hidden2'))
model2S.add(Dense(units=nout, activation='sigmoid', name='output'))
model2S.summary()

In [ ]:
model2S.compile(optimizer=opt,
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [ ]:
model2S.fit(Xtr_norm, ytrain, epochs=30, batch_size=100, validation_data=(Xts_norm,ytest))

In [ ]:
# model with two hidden layers with 12 hidden units each
# using sigmoid activation 
model3S = Sequential()
model3S.add(Dense(units=12, input_shape=(nin,), activation='sigmoid', name='hidden1'))
model3S.add(Dense(units=12, input_shape=(nin,), activation='sigmoid', name='hidden2'))
model3S.add(Dense(units=nout, activation='sigmoid', name='output'))
model3S.summary()

In [ ]:
model3S.compile(optimizer=opt,
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [ ]:
model3S.fit(Xtr_norm, ytrain, epochs=30, batch_size=100, validation_data=(Xts_norm,ytest))

In [ ]:
# model with two hidden layers with 16 hidden units each
# using sigmoid activation 
model4S = Sequential()
model4S.add(Dense(units=16, input_shape=(nin,), activation='sigmoid', name='hidden1'))
model4S.add(Dense(units=16, input_shape=(nin,), activation='sigmoid', name='hidden2'))
model4S.add(Dense(units=nout, activation='sigmoid', name='output'))
model4S.summary()

In [ ]:
model4S.compile(optimizer=opt,
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [ ]:
model4S.fit(Xtr_norm, ytrain, epochs=30, batch_size=100, validation_data=(Xts_norm,ytest))

In [ ]:
# model with two hidden layers with 20 hidden units each
# using sigmoid activation 
model5S = Sequential()
model5S.add(Dense(units=, input_shape=(nin,), activation='sigmoid', name='hidden1'))
model5S.add(Dense(units=20, input_shape=(nin,), activation='sigmoid', name='hidden2'))
model5S.add(Dense(units=nout, activation='sigmoid', name='output'))
model5S.summary()

In [ ]:
model5S.compile(optimizer=opt,
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [ ]:
model5S.fit(Xtr_norm, ytrain, epochs=30, batch_size=100, validation_data=(Xts_norm,ytest))

Accuracy doesn't seem to go over 62% with any of the models trained.
We did not include the identity of either the player or the closest defender. Would that play a role in increasing accuracy?


In [ ]:
data1 = pd.read_csv('Data/data.csv')

# separate data into matrix of predictor variables and vector of target variables
names1 = ['LOCATION', 'PLAYER_NAME', 'CLOSEST_DEFENDER','SHOT_NUMBER', 'PERIOD', 'GAME_CLOCK', 'SHOT_CLOCK',
         'DRIBBLES', 'SHOT_DIST', 'PTS_TYPE', 'CLOSE_DEF_DIST', 
         'PLAYER_HEIGHT','DEFENDER_HEIGHT', 'PLAYER_PTS', 'PLAYER_FG%', 'PLAYER_3P%',
          'PLAYER_FT%','DEFENDER_STL', 'DEFENDER_BLK', 'SHOT_RESULT']

# convert shot_result to either 0 or 1
data1['SHOT_RESULT'] = (data1['SHOT_RESULT'] == 'made').astype(int)

# convert game_clock to seconds
data1['GAME_CLOCK'] = data1['GAME_CLOCK'].apply(lambda x: convert_time(x))

# encode categorical data 
data_encoded = pd.get_dummies(data1[names1])

# split into train and test set with roughly 75-25 split
train, test = train_test_split(data_encoded, test_size=0.25)
Xtr1 = train.drop(['SHOT_RESULT'], axis=1)
ytr1 = train['SHOT_RESULT']
Xts1 = test.drop(['SHOT_RESULT'], axis=1)
yts1 = test['SHOT_RESULT']

In [ ]:
ytr1 = ytr1.values
yts1 = yts1.values

In [ ]:
# normalization
scaler = StandardScaler()
scaler.fit(Xtr1)
Xtr1 = scaler.transform(Xtr1)
scaler.fit(Xts1)
Xts1 = scaler.transform(Xts1)

In [ ]:
K.clear_session()

In [ ]:
Xtr1.shape

In [ ]:
nin = Xtr1.shape[1] # number of inputs 
nout = 1 # make or miss

In [ ]:
# model with two hidden layers with 100 hidden units each
# using sigmoid activation 
model100 = Sequential()
model100.add(Dense(units=100, input_shape=(nin,), activation='sigmoid', name='hidden1'))
model100.add(Dense(units=100, input_shape=(nin,), activation='sigmoid', name='hidden2'))
model100.add(Dense(units=nout, activation='sigmoid', name='output'))
model100.summary()

In [ ]:
model100.compile(optimizer=opt,
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [ ]:
model100.fit(Xtr1, ytr1, epochs=30, batch_size=100, validation_data=(Xts1,yts1))

Validation accuracy horrible while training accuracy goes up. Overfitting? Decrease hidden units. Let's start low again. From 16 let's say. And go up to ~50

In [ ]:
# model with two hidden layers with 16 hidden units each
# using sigmoid activation 
model16 = Sequential()
model16.add(Dense(units=16, input_shape=(nin,), activation='sigmoid', name='hidden1'))
model16.add(Dense(units=16, input_shape=(nin,), activation='sigmoid', name='hidden2'))
model16.add(Dense(units=nout, activation='sigmoid', name='output'))
model16.summary()

In [ ]:
model16.compile(optimizer=opt,
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [ ]:
model16.fit(Xtr1, ytr1, epochs=30, batch_size=100, validation_data=(Xts1,yts1))

In [ ]:
# model with two hidden layers with 12 hidden units each
# using sigmoid activation 
model12 = Sequential()
model12.add(Dense(units=12, input_shape=(nin,), activation='sigmoid', name='hidden1'))
model12.add(Dense(units=12, input_shape=(nin,), activation='sigmoid', name='hidden2'))
model12.add(Dense(units=nout, activation='sigmoid', name='output'))
model12.summary()

In [ ]:
model12.compile(optimizer=opt,
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [ ]:
model20.fit(Xtr1, ytr1, epochs=30, batch_size=100, validation_data=(Xts1,yts1))

In [ ]:
# model with two hidden layers with 12 hidden units each
# using sigmoid activation 
model12 = Sequential()
model12.add(Dense(units=12, input_shape=(nin,), activation='sigmoid', name='hidden1'))
model12.add(Dense(units=12, input_shape=(nin,), activation='sigmoid', name='hidden2'))
model12.add(Dense(units=nout, activation='sigmoid', name='output'))
model12.summary()

In [ ]:
model12.compile(optimizer=opt,
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [ ]:
model12.fit(Xtr1, ytr1, epochs=30, batch_size=100, validation_data=(Xts1,yts1))

reweight parameters 
try different number of neighbors
feature transformation 
- ratio of shot distance to shot defender, etc
- do we keep the original features 
try logistic regression 
 

In [ ]:
Xtrain_new['SHOT_DIST']/Xtrain_new['DEFENDER_BLK']

In [ ]:
plt.ylim(6)